In [2]:
import pandas as pd
import numpy as np
import re

from urllib.parse import urlparse
import contractions
import string
import num2words

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer
from nltk.corpus import words, stopwords
from nltk.metrics.distance import jaccard_distance, edit_distance
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [5]:
train_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/tanfiona CausalNewsCorpus master data-V2/train_subtask2.csv')
train_df.head()

,corpus,doc_id,sent_id,eg_id,index,text,text_w_pairs,seq_label,pair_label,context,num_sents
0,cnc,train_05_299,259,0,cnc_train_05_299_259_0,"ADILABAD : TRS condemns arrests March 11 , 201...","ADILABAD : TRS condemns arrests March 11 , 201...",1,1,NaN,1
1,cnc,train_09_B_24,2505,0,cnc_train_09_B_24_2505_0,"However , local villagers resisted the searche...","However , <ARG1>local villagers resisted the s...",1,1,NaN,1
2,cnc,train_02_109,291,0,cnc_train_02_109_291_0,"In 2009 , riots broke out in the capital , Uru...","In 2009 , <ARG0>riots broke out in the capital...",1,1,NaN,1
3,cnc,train_05_257,2109,0,cnc_train_05_257_2109_0,"The 60 - page charge sheet , filed by National...","The 60 - page charge sheet , filed by National...",1,1,NaN,1
4,cnc,train_05_223,1332,0,cnc_train_05_223_1332_0,TREATMENT Residents of Ravensmead on the Cape ...,TREATMENT <ARG1>Residents of Ravensmead on the...,1,1,NaN,1


In [4]:
train_grouped_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/tanfiona CausalNewsCorpus master data-V2/train_subtask2_grouped.csv')
train_grouped_df.head()

,corpus,doc_id,sent_id,eg_id,index,text,causal_text_w_pairs,num_rs
0,cnc,train_01_0,892,0,cnc_train_01_0_892_0,The State alleged they hacked Sabata Petros Ch...,['The State alleged <ARG1>they hacked Sabata P...,1
1,cnc,train_01_1,2714,0,cnc_train_01_1_2714_0,Chale was allegedly chased by a group of about...,[],0
2,cnc,train_01_10,2619,0,cnc_train_01_10_2619_0,The farmworkers ' strike resumed on Tuesday wh...,"[""<ARG1>The farmworkers ' strike resumed on Tu...",1
3,cnc,train_01_100,2680,0,cnc_train_01_100_2680_0,Demonstrators have filed for a permit to hold ...,['<ARG1>Demonstrators have filed for a permit<...,2
4,cnc,train_01_101,3090,0,cnc_train_01_101_3090_0,"Footage of the attack , which included a pregn...","['<ARG0>Footage of the attack , which included...",2


# Dev

In [6]:
dev_text_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/tanfiona CausalNewsCorpus master data-V2/dev_subtask2_text.csv')
dev_text_df.head()

,corpus,doc_id,sent_id,eg_id,index,text,causal_text_w_pairs,num_rs
0,cnc,train_10_0,2136,0,cnc_train_10_0_2136_0,The movement was catapulted into the headlines...,NaN,NaN
1,cnc,train_10_1,350,0,cnc_train_10_1_350_0,Several thousand protesters took to the street...,NaN,NaN
2,cnc,train_10_10,3104,0,cnc_train_10_10_3104_0,""" The protest is not just about saving Ms. Med...",NaN,NaN
3,cnc,train_10_100,1188,0,cnc_train_10_100_1188_0,Hong Kong Baptist University protest Contrite ...,NaN,NaN
4,cnc,train_10_100,1734,0,cnc_train_10_100_1734_0,"PUBLISHED : Saturday , 03 February , 2018 , 3:...",NaN,NaN


In [7]:
dev_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/tanfiona CausalNewsCorpus master data-V2/dev_subtask2.csv')
dev_df.head()

,corpus,doc_id,sent_id,eg_id,index,text,text_w_pairs,seq_label,pair_label,context,num_sents
0,cnc,train_10_196,284,0,cnc_train_10_196_284_0,The Sadtu protest was a call for the resignati...,<ARG1>The Sadtu protest</ARG1> was <ARG0><SIG0...,1,1,NaN,1
1,cnc,train_10_276,2598,0,cnc_train_10_276_2598_0,Tuen Mun Court Deputy Magistrate So Kai-cheung...,Tuen Mun Court Deputy Magistrate So Kai-cheung...,1,1,NaN,1
2,cnc,train_10_276,2598,1,cnc_train_10_276_2598_1,Tuen Mun Court Deputy Magistrate So Kai-cheung...,Tuen Mun Court Deputy Magistrate So Kai-cheung...,1,1,NaN,1
3,cnc,train_10_184,356,0,cnc_train_10_184_356_0,Two militants had also been killed in the fire...,<ARG1>Two militants had also been killed</ARG1...,1,1,NaN,1
4,cnc,train_10_210,985,0,cnc_train_10_210_985_0,The end to hunger strike came as a relief to t...,<ARG0>The end to hunger strike</ARG0> <SIG0>ca...,1,1,NaN,1


In [8]:
dev_grouped_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/tanfiona CausalNewsCorpus master data-V2/dev_subtask2_grouped.csv')
dev_grouped_df.head()

,corpus,doc_id,sent_id,eg_id,index,text,causal_text_w_pairs,num_rs
0,cnc,train_10_0,2136,0,cnc_train_10_0_2136_0,The movement was catapulted into the headlines...,['<ARG1>The movement was catapulted into the h...,1
1,cnc,train_10_1,350,0,cnc_train_10_1_350_0,Several thousand protesters took to the street...,['<ARG1>Several thousand protesters took to th...,2
2,cnc,train_10_10,3104,0,cnc_train_10_10_3104_0,""" The protest is not just about saving Ms. Med...",[],0
3,cnc,train_10_100,1188,0,cnc_train_10_100_1188_0,Hong Kong Baptist University protest Contrite ...,['Hong Kong Baptist University protest Contrit...,1
4,cnc,train_10_100,1734,0,cnc_train_10_100_1734_0,"PUBLISHED : Saturday , 03 February , 2018 , 3:...","['PUBLISHED : Saturday , 03 February , 2018 , ...",3
